# Visualization and Playback

### Imports

In [59]:
import mingus.containers as mc
from mingus.midi import midi_file_out
from IPython.display import Audio
from pretty_midi import PrettyMIDI
import pandas as pd
import numpy as np
import os
import mido

In [16]:
# read from json
filepath = 'data/chords/cleaned_data.json'
full_df = pd.read_json(filepath, orient='table')
full_df.head()

,artist_name,song_name,chords,genres,popularity
0,Justin Bieber,"10,000 Hours","[G, G/B, C, G, G/B, C, G, Em, C, G, Em, C, G, ...","['canadian pop', 'pop', 'post-teen pop']",100
1,Justin Bieber,2 Much,"[F#m7, D9, F#m7, D9, F#m7, D9, E, F#m7, A/C#, ...","['canadian pop', 'pop', 'post-teen pop']",100
2,Justin Bieber,2u (feat. David Guetta),"[Em, D, C, D, Em, D, C, D, Em, D, C, Am, D, Em...","['canadian pop', 'pop', 'post-teen pop']",100
3,Justin Bieber,All Around The World,"[Em, Bm, Am, C, Em, Bm, Am, C, Em, Bm, Am, C, ...","['canadian pop', 'pop', 'post-teen pop']",100
4,Justin Bieber,All Around The World (acoustic),"[Gm, Dm, C, Gm, Dm, C, Gm, Dm, C, Gm, A#, Cdim...","['canadian pop', 'pop', 'post-teen pop']",100


## Experimenting with mingus

In [17]:
root = 'C'

In [18]:
# chords.major_triad(root)
# chords.minor_triad(root)
# chords.diminished_triad(root)
# chords.augmented_triad(root)
# chords.suspended_triad(root)

In [19]:
# chords.from_shorthand('Cm7')

In [20]:
# chords_df = full_df['chords']

# song1 = []
# for ch in chords_df[0]:
#     song1.append(chords.from_shorthand(ch))
# song1[0:10]

## Experimenting with Fluidsynth

In [21]:
# sound1 = "soundfont_lib/Discarded_Wurlitzer_piano.sf2"
# sound2 = "soundfont_lib/YDP-GrandPiano-20160804.sf2"

In [22]:
# fs = fluidsynth.Synth()
# fs.start()

# sfid = fs.sfload(sound2)
# fs.program_select(0, sfid, 0, 0)

# fs.noteon(0, 60, 100)
# fs.noteon(0, 67, 100)
# fs.noteon(0, 76, 100)

# time.sleep(1.0)

# fs.noteoff(0, 60)
# fs.noteoff(0, 67)
# fs.noteoff(0, 76)

# time.sleep(1.0)
# fs.delete()

## Making Midi file

In [23]:
chords_df = full_df['chords']

c = chords_df[0][0:10]
c

['G', 'G/B', 'C', 'G', 'G/B', 'C', 'G', 'Em', 'C', 'G']

In [34]:
t = mc.Track().from_chords(c, 1)
t

[None, [[[0.0, 1, ['G-4', 'B-4', 'D-5']]], [[0.0, 1, ['B-4', 'G-5', 'B-5', 'D-6']]], [[0.0, 1, ['C-4', 'E-4', 'G-4']]], [[0.0, 1, ['G-4', 'B-4', 'D-5']]], [[0.0, 1, ['B-4', 'G-5', 'B-5', 'D-6']]], [[0.0, 1, ['C-4', 'E-4', 'G-4']]], [[0.0, 1, ['G-4', 'B-4', 'D-5']]], [[0.0, 1, ['E-4', 'G-4', 'B-4']]], [[0.0, 1, ['C-4', 'E-4', 'G-4']]], [[0.0, 1, ['G-4', 'B-4', 'D-5']]]]]

In [25]:
# midi_files = os.listdir("data/midi")
# if midi_files == []:
#     version = 1
# else:
#     version = int(midi_files[-1][-1]) + 1
# version
version = 1

In [26]:
midi_file_out.write_Track(f'data/midi/midi_{version}.mid', t)

True

## Playing midi (pyfluidsynth) [didn't work]

In [27]:
# midipath = os.listdir("data/midi/")[-1]
# midipath = 'data/midi/' + midipath
# midipath

In [28]:
# fs = fluidsynth.Synth()
# fs.start()

# sfid = fs.sfload(sound2)
# fs.program_select(0, sfid, 0, 0)

# fs.play_midi_file(midipath)
# fs.play_midi_stop()

## Playing midi (prettymidi)

In [29]:
sound1 = "soundfont_lib/Discarded_Wurlitzer_piano.sf2"
sound2 = "soundfont_lib/YDP-GrandPiano-20160804.sf2"

In [30]:
midipath = os.listdir("data/midi/")[-1]
midipath = 'data/midi/' + midipath
midipath

'data/midi/midi_1.mid'

In [57]:
music = PrettyMIDI(midi_file=midipath)
music

In [55]:
waveform = music.fluidsynth(44100.0, sound2)
audio_file = Audio(waveform, rate=44100)
audio_file

## Reading/Writing Audio Files

In [32]:
audiopath = f'data/audio/audio_{version}.mp3'
with open(audiopath, 'wb') as f:
    f.write(audio_file.data)

## Trying not to write files

In [40]:
from mingus.midi.midi_track import MidiTrack
from binascii import a2b_hex

In [41]:
class MidiFile(object):

    """A class that generates MIDI files from MidiTracks."""

    tracks = []
    time_division = b"\x00\x48"

    def __init__(self, tracks=None):
        if tracks is None:
            tracks = []
        self.reset()
        self.tracks = tracks

    def get_midi_data(self):
        """Collect and return the raw, binary MIDI data from the tracks."""
        tracks = [t.get_midi_data() for t in self.tracks if t.track_data != b""]
        return self.header() + b"".join(tracks)

    def header(self):
        """Return a header for type 1 MIDI file."""
        tracks = a2b_hex("%04x" % len([t for t in self.tracks if t.track_data != ""]))
        return b"MThd\x00\x00\x00\x06\x00\x01" + tracks + self.time_division

    def reset(self):
        """Reset every track."""
        [t.reset() for t in self.tracks]

    def write_file(self, file, verbose=False):
        """Collect the data from get_midi_data and write to file."""
        dat = self.get_midi_data()
        try:
            f = open(file, "wb")
        except:
            print("Couldn't open '%s' for writing." % file)
            return False
        try:
            f.write(dat)
        except:
            print("An error occured while writing data to %s." % file)
            return False
        f.close()
        if verbose:
            print("Written %d bytes to %s." % (len(dat), file))
        return True

In [52]:
track = mc.Track().from_chords(chords_df[0][0:10], 1)
repeat=0
m = MidiFile()
t = MidiTrack(120)
m.tracks = [t]
while repeat >= 0:
    t.play_Track(track)
    repeat -= 1
midi_data = m.get_midi_data()

In [53]:
signals = mido.parse_all(midi_data)

[Message('reset', time=0),
 Message('polytouch', channel=1, note=32, value=0, time=0),
 Message('reset', time=0),
 Message('reset', time=0),
 Message('reset', time=0),
 Message('note_on', channel=1, note=67, velocity=64, time=0),
 Message('note_on', channel=1, note=71, velocity=64, time=0),
 Message('note_on', channel=1, note=74, velocity=64, time=0),
 Message('note_off', channel=1, note=67, velocity=64, time=0),
 Message('note_off', channel=1, note=71, velocity=64, time=0),
 Message('note_off', channel=1, note=74, velocity=64, time=0),
 Message('reset', time=0),
 Message('reset', time=0),
 Message('note_on', channel=1, note=71, velocity=64, time=0),
 Message('note_on', channel=1, note=79, velocity=64, time=0),
 Message('note_on', channel=1, note=83, velocity=64, time=0),
 Message('note_on', channel=1, note=86, velocity=64, time=0),
 Message('note_off', channel=1, note=71, velocity=64, time=0),
 Message('note_off', channel=1, note=79, velocity=64, time=0),
 Message('note_off', channel=

In [51]:
music = PrettyMIDI(midi_file=m)

waveform = music.fluidsynth(44100.0, sound2)

audio_file = Audio(waveform, rate=44100)
audio_file

AttributeError: 'MidiFile' object has no attribute 'read'